In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ventillator-extrafeaturesfnn/ventilator_extraFeatures_fnn1.csv
/kaggle/input/ventillator-extrafeaturesfnn/__results__.html
/kaggle/input/ventillator-extrafeaturesfnn/ventilator_extraFeatures_fnn1.h5
/kaggle/input/ventillator-extrafeaturesfnn/__notebook__.ipynb
/kaggle/input/ventillator-extrafeaturesfnn/__output__.json
/kaggle/input/ventillator-extrafeaturesfnn/custom.css
/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
##Analyze the ventillator pressure dataset
import pandas as pd
import numpy as np
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

DEBUG = False

trainDataPath = "/kaggle/input/ventilator-pressure-prediction/train.csv"
df_data = pd.read_csv(trainDataPath, header=0)

df_data.head()
#df_data.describe()

testDataPath = "/kaggle/input/ventilator-pressure-prediction/test.csv"
df_test = pd.read_csv(testDataPath, header=0)

print(df_test.head())

if DEBUG:
    df_data = df_data.iloc[:80*100, :]
    df_test = df_test.iloc[:80*100, :]
print(df_data.shape)
print(df_test.shape)

   id  breath_id  R   C  time_step       u_in  u_out
0   1          0  5  20   0.000000   0.000000      0
1   2          0  5  20   0.031904   7.515046      0
2   3          0  5  20   0.063827  14.651675      0
3   4          0  5  20   0.095751  21.230610      0
4   5          0  5  20   0.127644  26.320956      0
(6036000, 8)
(4024000, 7)


In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
# df_data = df_data.iloc[0:240, :]
# df_data["u_in_prev1"] = df_data.groupby("breath_id")["u_in"].shift(1, fill_value=0)
# df_data["u_in_prev2"] = df_data.groupby("breath_id")["u_in"].shift(2, fill_value=0)
# df_data["u_in_prev3"] = df_data.groupby("breath_id")["u_in"].shift(3, fill_value=0)
# df_data["u_in_prev4"] = df_data.groupby("breath_id")["u_in"].shift(4, fill_value=0)
# df_data["u_in_prev5"] = df_data.groupby("breath_id")["u_in"].shift(5, fill_value=0)
# df_data["u_in_prev6"] = df_data.groupby("breath_id")["u_in"].shift(6, fill_value=0)
# df_data["u_in_next1"] = df_data.groupby("breath_id")["u_in"].shift(-1, fill_value=0)
# df_data["u_in_next2"] = df_data.groupby("breath_id")["u_in"].shift(-2, fill_value=0)
# df_data["u_in_next3"] = df_data.groupby("breath_id")["u_in"].shift(-3, fill_value=0)
# df_data["u_in_next4"] = df_data.groupby("breath_id")["u_in"].shift(-4, fill_value=0)
# df_data["u_in_next5"] = df_data.groupby("breath_id")["u_in"].shift(-5, fill_value=0)
# df_data["u_in_next6"] = df_data.groupby("breath_id")["u_in"].shift(-6, fill_value=0)
# df_data["u_in_cumm"] = df_data.groupby("breath_id")["u_in"].cumsum()
# df_data["area"] = df_data["time_step"]*df_data["u_in"]
# df_data["area"] = df_data.groupby("breath_id")["area"].cumsum()
# df_data["u_in_diff1"] = df_data["u_in"] - df_data["u_in_prev1"]
# df_data["u_in_diff2"] = df_data["u_in"] - df_data["u_in_prev2"]
# df_data["u_in_diff3"] = df_data["u_in"] - df_data["u_in_prev3"]
# df_data["u_in_diff4"] = df_data["u_in"] - df_data["u_in_prev4"]
# df_data["u_in_diff5"] = df_data["u_in"] - df_data["u_in_prev5"]
# df_data["u_in_diff6"] = df_data["u_in"] - df_data["u_in_prev6"]

# df_data["u_out_prev1"] = df_data.groupby("breath_id")["u_out"].shift(1, fill_value=0)
# df_data["u_out_prev2"] = df_data.groupby("breath_id")["u_out"].shift(2, fill_value=0)
# df_data["u_out_prev3"] = df_data.groupby("breath_id")["u_out"].shift(3, fill_value=0)
# df_data["u_out_prev4"] = df_data.groupby("breath_id")["u_out"].shift(4, fill_value=0)
# df_data["u_out_prev5"] = df_data.groupby("breath_id")["u_out"].shift(5, fill_value=0)
# df_data["u_out_prev6"] = df_data.groupby("breath_id")["u_out"].shift(6, fill_value=0)
# df_data["u_out_next1"] = df_data.groupby("breath_id")["u_out"].shift(-1, fill_value=0)
# df_data["u_out_next2"] = df_data.groupby("breath_id")["u_out"].shift(-2, fill_value=0)
# df_data["u_out_next3"] = df_data.groupby("breath_id")["u_out"].shift(-3, fill_value=0)
# df_data["u_out_next4"] = df_data.groupby("breath_id")["u_out"].shift(-4, fill_value=0)
# df_data["u_out_next5"] = df_data.groupby("breath_id")["u_out"].shift(-5, fill_value=0)
# df_data["u_out_next6"] = df_data.groupby("breath_id")["u_out"].shift(-6, fill_value=0)
# df_data["u_out_diff1"] = df_data["u_out"] - df_data["u_out_prev1"]
# df_data["u_out_diff2"] = df_data["u_out"] - df_data["u_out_prev2"]
# df_data["u_out_diff3"] = df_data["u_out"] - df_data["u_out_prev3"]
# df_data["u_out_diff4"] = df_data["u_out"] - df_data["u_out_prev4"]
# df_data["u_out_diff5"] = df_data["u_out"] - df_data["u_out_prev5"]
# df_data["u_out_diff6"] = df_data["u_out"] - df_data["u_out_prev6"]

# df_data["u_in_diff_max"] = df_data.groupby("breath_id")["u_in"].transform("max") - df_data["u_in"]
# df_data["u_in_diff_mean"] = df_data.groupby("breath_id")["u_in"].transform("mean") - df_data["u_in"]

# df_data["u_in_u_out"] = df_data["u_in"] * df_data["u_out"]
# df_data["u_out_timestep"] = df_data["u_out"] * df_data["time_step"]

# df_data['R'] = df_data['R'].astype(str)
# df_data['C'] = df_data['C'].astype(str)
# df_data['R__C'] = df_data["R"].astype(str) + '__' + df_data["C"].astype(str)
# df_data = pd.get_dummies(df_data)

# print(df_data.head())
# print(df_data.iloc[75:85,:])


In [5]:
def addFeatures(df_data):
    df_data["u_in_prev1"] = df_data.groupby("breath_id")["u_in"].shift(1, fill_value=0)
    df_data["u_in_prev2"] = df_data.groupby("breath_id")["u_in"].shift(2, fill_value=0)
    df_data["u_in_prev3"] = df_data.groupby("breath_id")["u_in"].shift(3, fill_value=0)
    df_data["u_in_prev4"] = df_data.groupby("breath_id")["u_in"].shift(4, fill_value=0)
    df_data["u_in_prev5"] = df_data.groupby("breath_id")["u_in"].shift(5, fill_value=0)
    df_data["u_in_prev6"] = df_data.groupby("breath_id")["u_in"].shift(6, fill_value=0)
    df_data["u_in_next1"] = df_data.groupby("breath_id")["u_in"].shift(-1, fill_value=0)
    df_data["u_in_next2"] = df_data.groupby("breath_id")["u_in"].shift(-2, fill_value=0)
    df_data["u_in_next3"] = df_data.groupby("breath_id")["u_in"].shift(-3, fill_value=0)
    df_data["u_in_next4"] = df_data.groupby("breath_id")["u_in"].shift(-4, fill_value=0)
    df_data["u_in_next5"] = df_data.groupby("breath_id")["u_in"].shift(-5, fill_value=0)
    df_data["u_in_next6"] = df_data.groupby("breath_id")["u_in"].shift(-6, fill_value=0)
    df_data["u_in_cumm"] = df_data.groupby("breath_id")["u_in"].cumsum()
    df_data["area"] = df_data["time_step"]*df_data["u_in"]
    df_data["area"] = df_data.groupby("breath_id")["area"].cumsum()
    df_data["u_in_diff1"] = df_data["u_in"] - df_data["u_in_prev1"]
    df_data["u_in_diff2"] = df_data["u_in"] - df_data["u_in_prev2"]
    df_data["u_in_diff3"] = df_data["u_in"] - df_data["u_in_prev3"]
    df_data["u_in_diff4"] = df_data["u_in"] - df_data["u_in_prev4"]
    df_data["u_in_diff5"] = df_data["u_in"] - df_data["u_in_prev5"]
    df_data["u_in_diff6"] = df_data["u_in"] - df_data["u_in_prev6"]

    df_data["u_out_prev1"] = df_data.groupby("breath_id")["u_out"].shift(1, fill_value=0)
    df_data["u_out_prev2"] = df_data.groupby("breath_id")["u_out"].shift(2, fill_value=0)
    df_data["u_out_prev3"] = df_data.groupby("breath_id")["u_out"].shift(3, fill_value=0)
    df_data["u_out_prev4"] = df_data.groupby("breath_id")["u_out"].shift(4, fill_value=0)
    df_data["u_out_prev5"] = df_data.groupby("breath_id")["u_out"].shift(5, fill_value=0)
    df_data["u_out_prev6"] = df_data.groupby("breath_id")["u_out"].shift(6, fill_value=0)
    df_data["u_out_next1"] = df_data.groupby("breath_id")["u_out"].shift(-1, fill_value=0)
    df_data["u_out_next2"] = df_data.groupby("breath_id")["u_out"].shift(-2, fill_value=0)
    df_data["u_out_next3"] = df_data.groupby("breath_id")["u_out"].shift(-3, fill_value=0)
    df_data["u_out_next4"] = df_data.groupby("breath_id")["u_out"].shift(-4, fill_value=0)
    df_data["u_out_next5"] = df_data.groupby("breath_id")["u_out"].shift(-5, fill_value=0)
    df_data["u_out_next6"] = df_data.groupby("breath_id")["u_out"].shift(-6, fill_value=0)
    df_data["u_out_diff1"] = df_data["u_out"] - df_data["u_out_prev1"]
    df_data["u_out_diff2"] = df_data["u_out"] - df_data["u_out_prev2"]
    df_data["u_out_diff3"] = df_data["u_out"] - df_data["u_out_prev3"]
    df_data["u_out_diff4"] = df_data["u_out"] - df_data["u_out_prev4"]
    df_data["u_out_diff5"] = df_data["u_out"] - df_data["u_out_prev5"]
    df_data["u_out_diff6"] = df_data["u_out"] - df_data["u_out_prev6"]

    df_data["u_in_diff_max"] = df_data.groupby("breath_id")["u_in"].transform("max") - df_data["u_in"]
    df_data["u_in_diff_mean"] = df_data.groupby("breath_id")["u_in"].transform("mean") - df_data["u_in"]

    df_data["u_in_u_out"] = df_data["u_in"] * df_data["u_out"]
    df_data["u_out_timestep"] = df_data["u_out"] * df_data["time_step"]

    df_data['R'] = df_data['R'].astype(str)
    df_data['C'] = df_data['C'].astype(str)
    df_data['R__C'] = df_data["R"].astype(str) + '__' + df_data["C"].astype(str)
    df_data = pd.get_dummies(df_data)
    return df_data

In [6]:
##Added features for data and test
df_data = addFeatures(df_data)
df_test = addFeatures(df_test)
print(df_data.shape)
print(df_test.shape)

(6036000, 63)
(4024000, 62)


In [7]:
##Reduce memory data occupied by changing the datatypes
df_data = reduce_mem_usage(df_data)
#print(df_data.dtypes)
df_test = reduce_mem_usage(df_test)
#df_test.dtypes

Mem. usage decreased from 2296.80 Mb to 863.46 Mb (62.4% reduction)
Mem. usage decreased from 1500.50 Mb to 560.29 Mb (62.7% reduction)


In [8]:
##Shuffle the data and get the features similar for train and test
df_data = df_data.sample(frac=1)
df_y_data = df_data["pressure"].copy()
df_data = df_data.drop(["id", "breath_id", "pressure"], axis=1)

df_test = df_test.drop(["id", "breath_id"], axis=1)
print("data shape= %s, Test shape= %s" %(df_data.shape, df_test.shape))

data shape= (6036000, 60), Test shape= (4024000, 60)


In [9]:
##Normalize the data
from sklearn.preprocessing import RobustScaler, normalize
RS = RobustScaler()
train = RS.fit_transform(df_data)
test = RS.transform(df_test)
print(train[0,:])
print(test[0,:])

[ 0.08128715 -0.95553905  0.         -0.8641206  -0.8425267  -0.8192579
 -0.7941516  -0.76915    -0.7384456  -0.8944703  -0.80248475 -0.63450694
 -0.49158016 -0.36766803 -0.25148168 -0.45099223 -0.21418367 -0.04551295
 -0.04861012 -0.05188134 -0.05116878 -0.05860326 -0.056019    0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.         -0.02694016
  0.25912455  0.          0.05469879  0.          0.          1.
  1.          0.          0.          0.          0.          0.
  1.          0.          0.          0.          0.          0.        ]
[-0.9890522  -0.95553905 -1.         -0.8641206  -0.8425267  -0.8192579
 -0.7941516  -0.76915    -0.7384456   0.6527179   2.085062    3.4264958
  4.4720716   5.33336     5.980515   -0.72544724 -0.5165805  -0.04551295
 -0.04861012 -0.05188134 -0.05116878 -0.05860326 -0.056019   -1.
 -1.         -1.         -1.

In [10]:
##Shuffle df_data and then get df_y_train and df_x_train
from sklearn.model_selection import train_test_split

## Remove random_state if we want variable train and dev sizes
#x_train, x_dev = train_test_split(train, train_size=0.8, random_state=9)
x_train, x_dev, y_train, y_dev = train_test_split(df_data, df_y_data, train_size=0.8)

df_data_y_sort = np.sort(df_y_data.unique())
print(len(df_data_y_sort))

del(df_data)
# del(df_y_data)
print("train size= %s, dev size = %s\n" %(x_train.shape, x_dev.shape))

950
train size= (4828800, 60), dev size = (1207200, 60)



In [11]:
##Create FNN for training
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping

##Clear models
tf.keras.backend.clear_session()

inputNodes = x_train.shape[1]

model = Sequential()
model.add(tf.keras.Input(shape=(inputNodes,)))
model.add(Dense(256,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               15616     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 442,753
Trainable params: 442,753
Non-trainable params: 0
_________________________________________________________________



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [12]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True,\
                                             verbose=1)

##Fit model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])

##Train the model
history = model.fit(x=x_train, y=y_train, batch_size=1024, epochs=100, validation_data=(x_dev, y_dev),\
                   callbacks=[earlystop])

2021-10-23 17:23:07.849267: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
4716/4716 [==============================] - 149s 31ms/step - loss: 1.0990 - mean_absolute_error: 1.0990 - val_loss: 0.8659 - val_mean_absolute_error: 0.8659
Epoch 2/100
4716/4716 [==============================] - 152s 32ms/step - loss: 0.6836 - mean_absolute_error: 0.6836 - val_loss: 0.6241 - val_mean_absolute_error: 0.6241
Epoch 3/100
4716/4716 [==============================] - 152s 32ms/step - loss: 0.6198 - mean_absolute_error: 0.6198 - val_loss: 0.5982 - val_mean_absolute_error: 0.5982
Epoch 4/100
4716/4716 [==============================] - 157s 33ms/step - loss: 0.5808 - mean_absolute_error: 0.5808 - val_loss: 0.5469 - val_mean_absolute_error: 0.5469
Epoch 5/100
4716/4716 [==============================] - 153s 32ms/step - loss: 0.5546 - mean_absolute_error: 0.5546 - val_loss: 0.5332 - val_mean_absolute_error: 0.5332
Epoch 6/100
4716/4716 [==============================] - 156s 33ms/step - loss: 0.5346 - mean_absolute_error: 0.5346 - val_loss: 0.5209 - val_mean_abs

In [13]:
model.save("./ventilator_extraFeatures_fnn1_lossMAE.h5")
##Evaluate model on dev set
score = model.evaluate(x_dev, y_dev, batch_size=10240)
score

118/118 [==============================] - 8s 67ms/step - loss: 0.4037 - mean_absolute_error: 0.4037


[0.40368565917015076, 0.40368565917015076]

In [14]:
df_sub = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/sample_submission.csv", header=0)
df_sub["pressure"] = model.predict(df_test, batch_size=10240)
df_sub.to_csv("./ventilator_extraFeatures_prev6_fnn1_lossMAE.csv", index=False)
print("Done saving file")

Done saving file


In [15]:
##Rounding off the predictions to closest from train dataset
import pandas as pd
import numpy as np

# df_data_y_sort = np.sort(df_data.iloc[:,-1].unique())
# print(len(df_data_y_sort))

def searchNearest(prediction):
    idx = np.searchsorted(df_data_y_sort, prediction)
    #print("prediction= %s, idx= %s" %(prediction,  idx))
    if (idx >= df_data_y_sort.shape[0]-1):
        ## If the number is greater than the largest value in sort array, return the largest element
        return df_data_y_sort[-1]
    
    lowerVal = df_data_y_sort[idx]
    upperVal = df_data_y_sort[idx+1]
    return lowerVal if np.abs(lowerVal - prediction) < np.abs(upperVal - prediction) else upperVal

#test_sub.drop("new_pressure", axis=1, inplace=True)
#test_sub = pd.read_csv("/kaggle/input/ventillator-extrafeaturesfnn/ventilator_extraFeatures_fnn1.csv", header=0)
test_sub = pd.read_csv("./ventilator_extraFeatures_prev6_fnn1_lossMAE.csv", header=0)
test_sub["pressure"] = test_sub["pressure"].apply(searchNearest)
print(test_sub.head())
test_sub.to_csv("./ventilator_extraFeatures_prev6_fnn1_lossMAE_round.csv", index=False)
print("Done saving test prediction file")

   id  pressure
0   1  6.118701
1   2  6.189003
2   3  7.102931
3   4  7.595046
4   5  9.001088
Done saving test prediction file
